In [2]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool.invoke("Who's the current President of France?")
print(results)

The current president is Emmanuel Macron, who succeeded François Hollande on 14 May 2017 following the 2017 presidential election, and was inaugurated for a second term on 7 May 2022 following the 2022 presidential election. Emmanuel Jean-Michel Frédéric Macron[b] (born 21 December 1977) is a French politician who has served as President of France and Co-Prince of Andorra since 2017. He served as Minister of the Economy, Industry, and Digital Affairs under President François Hollande from 2014 to 2016. Emmanuel Macron is currently serving as President of France . He took office upon winning the 2017 French presidential election. Macron was the founding member of Renaissance. He defeated National Rally nominee Marine Le Pen. He again defeated Le Pen in the 2022 French presidential election. The current president is Emmanuel Macron , who succeeded François Hollande on 14 May 2017 following the 2017 presidential election , and was ... François Gérard Georges Hollande (born 12 August 1954)

In [10]:
from langchain.tools import Tool
import random

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location."""
    # Dummy weather data
    weather_conditions = [
        {"condition": "Rainy", "temp_c": 15},
        {"condition": "Clear", "temp_c": 25},
        {"condition": "Windy", "temp_c": 20}
    ]
    # Randomly select a weather condition
    data = random.choice(weather_conditions)
    return f"Weather in {location}: {data['condition']}, {data['temp_c']}°C"

# Initialize the tool
weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location."
)

# Example usage
print(weather_info_tool.invoke("Taipei")) 

Weather in Taipei: Rainy, 15°C


In [11]:
from langchain.tools import Tool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        # List models from the specified author, sorted by downloads
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"

# Initialize the tool
hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

# Example usage
print(hub_stats_tool.invoke("facebook")) # Example: Get the most downloaded model by Facebook

The most downloaded model by facebook is facebook/opt-125m with 10,128,861 downloads.


In [17]:
import os
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

SYSTEM_POLICY = """You are Alfred, a multi-tool assistant.
Always CALL the appropriate tool(s) first, then answer ONLY from tool outputs.

Tools:
- search_tool — general web/entity lookups (companies, people, definitions).
- hub_stats_tool — popularity/leaderboard/stats for AI models/orgs on the model hub.
- weather_info_tool — weather/forecast for a specific location and date.

Answering rules:
- Summarize STRICTLY from tool outputs; do NOT add outside facts.
- Use your own words (don’t copy tool text verbatim).
- If tools return nothing relevant, reply exactly: I couldn’t find this with the available tools.

Style:
- Natural, friendly, concise. 1–2 sentences, no bullets or headings.
- Write in the user’s language.
"""

# Generate the chat interface, including the tools
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-Coder-32B-Instruct",
    huggingfacehub_api_token=os.getenv("HUGGINGFACE_API_KEY"),
)

chat = ChatHuggingFace(llm=llm, verbose=True)
tools = [search_tool, weather_info_tool, hub_stats_tool]
chat_with_tools = chat.bind_tools(tools)

# Generate the AgentState and Agent graph
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

def assistant(state: AgentState):
    msgs = [SystemMessage(content=SYSTEM_POLICY)] + state["messages"]
    return {
        "messages": [chat_with_tools.invoke(msgs)],
    }

## The graph
builder = StateGraph(AgentState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message requires a tool, route to tools
    # Otherwise, provide a direct response
    tools_condition,
)
builder.add_edge("tools", "assistant")
alfred = builder.compile()

messages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]
response = alfred.invoke({"messages": messages})

print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

🎩 Alfred's Response:
Facebook, now known as Meta, is an American online social media and social networking service company founded in 2004. Their most popular model on the model hub is OPT-125m, with over 10 million downloads.
